In [148]:
from collections import defaultdict
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem.porter import *
from sklearn import linear_model
from sets import Set

comments = []
usernames = []
#stop_words = set(stopwords.words('english'))
# Alternatively, just remove the basic stop words...
stop_words = {"a","an","and","are","as","at","be","by","for","from","has","he","in","is","it"
             ,"its","of","on","that","the","to","was","where","will","with"}
should_stem = 0
punctuation = set(string.punctuation)
stemmer = PorterStemmer()

def stem(w):
    if(should_stem == 0):
        return w
    else:
        return stemmer.stem(w)
    
#creating a small test file for convenience
def read(f):
    for l in open(f):
        yield eval(l)
        
x = open("opt_dataset/opt.json")
i = 0
training = open("opt_dataset/small_opt.json",'w')

for l in x:
    if (i>=0) and (i<20000):
        training.write(l)
        if(i==0):
            print l
    i += 1
training.close()

# forming the initial vocabulary on the training set of 600 samples
# Training set: 
# Test set: 
# Not using stemming currently, need to modify later


for i in read('opt_dataset/small_opt.json'):
    comments.append(i['comment'])
    usernames.append(i['userName'])

def bigrams(words):
    if(should_stem == 0):
        return nltk.bigrams(words)
    else:
        for i in range(len(words)):
            words[i] = stem(words[i])
        return nltk.bigrams(words)

        
# extracing all the known labels
labels = []
for l in open('opt_dataset/labels_1-900.txt'):
    if(eval(l) == 1):
        labels.append(1)
    else:
        labels.append(0)

{'userName': 'Roxanne Liu', 'recievedDate': '11/08/2015', 'docID': 'ICEB-2015-0002-12058', 'comment': 'It is very enssential for America to keep the international students. They are epquiped with skills and knowledge. The development of America relies on talents. Competition should not be the reason to ask them to leave.', 'postedDate': '11/11/2015'}



In [128]:
# Multinomial naive bayes!
def fit_params(num_examples, data_labels,update_vocab):
    train_no = num_examples
    # extracting the vocab
    if(update_vocab == 1):
        vocab_no = train_no
    else:
        vocab_no = 700
        
    vocab = Set()
    for i in range(vocab_no):
        r = ''.join([c for c in comments[i].lower() if not c in punctuation])
        for w in r.split():
            if(w not in stop_words):
                w = stem(w)
                vocab.add(w)

    pos_wordCount_mult = defaultdict(int)
    neg_wordCount_mult = defaultdict(int)
    
    for w in vocab:
        pos_wordCount_mult[w] = 0
        neg_wordCount_mult[w] = 0
    numberpos_mult = 0.0
    numberneg_mult = 0.0
    posexamples = 0.0
    negexamples = 0.0
    # calculating the probabilites
    for i in range(train_no):
        if(data_labels[i] == 1):
            posexamples += 1
        else:
            negexamples += 1
        r = ''.join([c for c in comments[i].lower() if not c in punctuation])
        bag = r.split()
        for w in bag:
            w = stem(w)
            if(w in vocab):
                if(data_labels[i] == 1):
                    pos_wordCount_mult[w] += 1
                    numberpos_mult += 1
                else:
                    neg_wordCount_mult[w] += 1
                    numberneg_mult += 1
                    
    posprob = posexamples/(posexamples + negexamples)
    negprob = negexamples/(posexamples + negexamples)
    
    pos_wordprob_mult = defaultdict(float)
    neg_wordprob_mult = defaultdict(float)

    for w in vocab:
        pos_wordprob_mult[w] = 1.0*(pos_wordCount_mult[w] + 1)/(len(vocab) + numberpos_mult)
        neg_wordprob_mult[w] = 1.0*(neg_wordCount_mult[w] + 1)/(len(vocab) + numberneg_mult)
    
    return vocab, pos_wordprob_mult,neg_wordprob_mult,posprob, negprob

import math
def predpos_mult(words,vocab,prob, frac):
    sum = 0.0
    for w in words:
        w = stem(w)
        if(w in vocab):
            sum += math.log(prob[w])
    return (math.log(frac) + sum)
            
def predneg_mult(words, vocab, prob, frac):
    sum = 0.0
    for w in words:
        w = stem(w)
        if(w in vocab):
            sum += math.log(prob[w])
    return (math.log(frac) + sum)



In [129]:
# Multinomial naive bayes bigrams!
def fit_params_bi(num_examples, data_labels,update_vocab):
    train_no = num_examples
    # extracting the bigrams
    if(update_vocab == 1):
        vocab_no = train_no
    else:
        vocab_no = 700

    all_bi = defaultdict(int)
    for i in range(vocab_no):
        r = ''.join([c for c in comments[i].lower() if not c in punctuation])
        for pair in bigrams(r.split()):
            all_bi[pair]=1
                
    pos_wordCount_mult_bi = defaultdict(int)
    neg_wordCount_mult_bi = defaultdict(int)
    
    for pair in all_bi:
        pos_wordCount_mult_bi[pair] = 0
        neg_wordCount_mult_bi[pair] = 0
    numberpos_mult_bi = 0.0
    numberneg_mult_bi = 0.0
    posexamples = 0.0
    negexamples = 0.0
    # calculating the probabilites
    for i in range(train_no):
        if(data_labels[i] == 1):
            posexamples += 1
        else:
            negexamples += 1
        r = ''.join([c for c in comments[i].lower() if not c in punctuation])
        bag = r.split()
        for pair in bigrams(bag):
            if(pair in all_bi):
                if(data_labels[i] == 1):
                    pos_wordCount_mult_bi[pair] += 1
                    numberpos_mult_bi += 1
                else:
                    neg_wordCount_mult_bi[pair] += 1
                    numberneg_mult_bi += 1
                    
    posprob = posexamples/(posexamples + negexamples)
    negprob = negexamples/(posexamples + negexamples)
    
    pos_wordprob_mult_bi = defaultdict(float)
    neg_wordprob_mult_bi = defaultdict(float)

    for pair in all_bi:
        pos_wordprob_mult_bi[pair] = 1.0*(pos_wordCount_mult_bi[pair] + 1)/(len(all_bi) + numberpos_mult_bi)
        neg_wordprob_mult_bi[pair] = 1.0*(neg_wordCount_mult_bi[pair] + 1)/(len(all_bi) + numberneg_mult_bi)
    
    return all_bi, pos_wordprob_mult_bi,neg_wordprob_mult_bi,posprob, negprob


import math
def predpos_mult_bi(words,vocab,prob, frac):
    sum = 0.0
    for pair in bigrams(words):
        if(pair in vocab):
            sum += math.log(prob[pair])
    return (sum)
            
def predneg_mult_bi(words, vocab, prob, frac):
    sum = 0.0
    for pair in bigrams(words):
        if(pair in vocab):
            sum += math.log(prob[pair])
    return (sum)



In [106]:

def train_NB(num_examples,data_labels,update_vocab):
    vocab, pp, nn, p, n = fit_params(num_examples,data_labels,update_vocab)
    all_bi, pp_bi, nn_bi, p, n = fit_params_bi(num_examples,data_labels,update_vocab)
    predictions_mult = []
    for i in range(len(comments)):
        r = ''.join([c for c in comments[i].lower() if not c in punctuation])
        words = r.split()
        x = predpos_mult(words,vocab,pp,p) #+ predpos_mult_bi(words,all_bi,pp_bi,p)
        y = predneg_mult(words,vocab,nn,n) #+ predneg_mult_bi(words,all_bi,nn_bi,n)
        if(x > y):
            predictions_mult.append(1)
        else:
            predictions_mult.append(0)
    return predictions_mult


In [150]:
# functions for accuracy prediction...
def print_accuracies(predictions, no_train,verbatim):
    train_accuracy = 0.0
    test_accuracy = 0.0
    valid_accuracy = 0.0
    tnrate = 0.0
    tprate = 0.0
    neg_examples = 0.0
    for i in range(900):
        if(i>=700 and labels[i] == 0):
            neg_examples += 1
        if(predictions[i] == labels[i]):
            if(labels[i] == 0 and i>=700):
                tnrate += 1
            if(labels[i] == 1 and i>=700):
                tprate += 1
            if i<600:
                train_accuracy += 1
            elif i<700:
                valid_accuracy += 1
            else:
                test_accuracy += 1
                
        elif(i>=700  and i<900 and verbatim == 1):
            print "\n", "comment #:",i,comments[i], "True label:",labels[i],"Predicted label:",predictions[i]
    print "\n", "Train accuracy:", train_accuracy/no_train, "Validation accuracy:", valid_accuracy/(100),\
    "Test accuracy:",test_accuracy/(200),"TNR:",(tnrate + 0.0001)/(neg_examples+0.0001), "TPR:", (tprate + 0.0001)/(200-neg_examples+0.0001)

In [141]:
predictions = train_NB(600,labels,1)
print_accuracies(predictions,600,0)


Train accuracy: 0.98 Validationa accuracy: 0.86 Test accuracy: 0.905 True negative rate: 0.407409602187


In [142]:
# Semi supervised learning, Some kind of expectation maximization...
# Refer to https://en.wikipedia.org/wiki/Naive_Bayes_classifier#Semi-supervised_parameter_estimation
# running the iterative process a fixed number of times.

num_iter = 1

while(num_iter < 10):
    print "iteration number:",num_iter
    predictions = train_NB(len(comments),predictions,0)
    print_accuracies(predictions,600,0)
    num_iter += 1


iteration number: 1

Train accuracy: 0.98 Validationa accuracy: 0.92 Test accuracy: 0.94 True negative rate: 0.703704801093
iteration number: 2

Train accuracy: 0.981666666667 Validationa accuracy: 0.96 Test accuracy: 0.96 True negative rate: 0.88888930041
iteration number: 3

Train accuracy: 0.975 Validationa accuracy: 0.97 Test accuracy: 0.965 True negative rate: 0.925926200273
iteration number: 4

Train accuracy: 0.975 Validationa accuracy: 0.97 Test accuracy: 0.965 True negative rate: 0.925926200273
iteration number: 5

Train accuracy: 0.975 Validationa accuracy: 0.97 Test accuracy: 0.965 True negative rate: 0.925926200273
iteration number: 6

Train accuracy: 0.975 Validationa accuracy: 0.97 Test accuracy: 0.965 True negative rate: 0.925926200273
iteration number: 7

Train accuracy: 0.973333333333 Validationa accuracy: 0.97 Test accuracy: 0.965 True negative rate: 0.925926200273
iteration number: 8

Train accuracy: 0.973333333333 Validationa accuracy: 0.97 Test accuracy: 0.965 True

In [151]:
print_accuracies(predictions,600,1)


comment #: 718 OPT is helping to find better workers for the jobs, not simply give the jobs to foreigners. True label: 1 Predicted label: 0

comment #: 765 Everyone should be allowed a green card so they can start paying taxes and be tracked True label: 1 Predicted label: 0

comment #: 806 The Optional Practical Training program allows U.S. employers to hire foreign students for up to 1 year or up to 29 months in certain fields of study. The program was meant to provide foreign students with on-the-job . True label: 1 Predicted label: 0

comment #: 818 I oppose the extension of OPT. Schools, especially public schools, welcome foreign students because they pay high tuition. And then, with extension of OPT, they earn back what they invest and maybe much more. Who is the winner? Obviously, foreign students who get more than they invested, schools which get a lot of money and companies which get a lot of comparable cheap workers. Who is the loser? Obviously not the government, the working